In [1]:
import pandas as pd
import MeCab
from collections import Counter, defaultdict

In [2]:
tagger = MeCab.Tagger("-Ochasen -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd -u /Users/yuki/foo/bar/user_dic2.dic")
tagger.parse("")

'EOS\n'

In [223]:
stop_words = ["♦","▪","⚪","・","◾",]

In [219]:
def tokenize_ja(text):
    node = tagger.parseToNode(str(text)) #分かち書きしたのが入っている。イテレータ
    while node:
        surface = node.surface.lower() #surfaceは単語の見た目　例：トろ
        feature = node.feature.split(",")
        hinshi = feature[0]
        genkei = feature[6]
        if surface not in stop_words and surface != "" and hinshi in ["形容詞", "動詞", "副詞", "助動詞", "接続詞", "連体詞", "感動詞", "接頭詞"]:
            yield genkei
        elif surface not in stop_words and surface != "" and hinshi == "名詞":
            yield surface
        node = node.next

In [218]:
def tokenize_ja_2(text):
    node = tagger.parseToNode(str(text)) #分かち書きしたのが入っている。イテレータ
    while node:
        surface = node.surface.lower() #surfaceは単語の見た目　例：トろ
        feature = node.feature.split(",")
        hinshi = feature[0]
        genkei = feature[6]
        if surface not in stop_words and surface != "" and hinshi in ["形容詞", "動詞", "副詞", "助動詞", "接続詞", "連体詞", "感動詞", "接頭詞"]:
            yield (genkei, hinshi)
        elif surface not in stop_words and surface != "" and hinshi == "名詞":
            yield (surface, hinshi)
        node = node.next

In [29]:
def tokenize(content):
    return [token for token in tokenize_ja(content)]

In [52]:
def tokenize_2(content):
    return [token for token in tokenize_ja_2(content)]

In [254]:
def output_df(file, jiku_list, n, t, s):
    store_name = file.replace("../pretest_", "").replace(".csv", "")
    df = pd.read_csv(file)
    num_word = 0 #全単語数を保存
    word_dict = defaultdict(int) #登場単語とその回数を保存
    if "dinner" in file:
        all_words = all_review_word_dinner_2 #他の店の登場単語とその回数の辞書
        all_num_words = num_word_dinner_2
        review_list = df["dinner_review"]
        for review in review_list:
            li = tokenize(review)
            for i in li:
                word_dict[i] += 1
                num_word += 1
    elif "lunch" in file:
        all_words = all_review_word_lunch_2 #他の店の登場単語とその回数の辞書
        all_num_words = num_word_lunch_2
        review_list = df["lunch_review"]
        for review in review_list:
            li = tokenize(review)
            for i in li:
                word_dict[i] += 1
                num_word += 1
    word_info = [] #軸に関する単語の出力情報を渡す のちにDataFrame化する
    for jiku_word_list in jiku_list:
        for ii in range(1, len(jiku_word_list)):
            w = jiku_word_list[ii]
            if word_dict[w]/num_word > (all_words[w]/all_num_words) * t:
                #他の店よりwという単語がt倍登場している時
                kyouki_pare = n_gram_result(w, n, review_list) #n_gramを使って出した共起度
                pare_word_list = [i[0][1] for i in kyouki_pare]
                pare_word_list.extend(["*" for i in range(s)])
                pare_word_time_list = [i[1] for i in kyouki_pare]
                pare_word_time_list.extend(["*" for i in range(s)])
                l = [jiku_word_list[0],
                     w,
                     word_dict[w], 
                    int(10000*word_dict[w]/num_word)/100,
                    int(100*word_dict[w]/len(review_list))/100,
                    int(10000*all_words[w]/all_num_words)/100,
                     ]
                l.extend([pare_word_list[i] +"　" +str(pare_word_time_list[i]) for i in range(s)])
                word_info.append(l)
    columns = ["軸","単語","登場数","登場率%/"+str(num_word),"レビューあたり/"+str(len(review_list)),"他のレビューでの登場率%"]
    columns.extend([i+1 for i in range(s)])
    df = pd.DataFrame(word_info, columns=columns)
    return df

In [238]:
def output_csv(file, n, t, s):
    store_name=file.replace("../pretest_", "").replace(".csv", "")
    df = output_df(file,jiku_list,n, t, s)
    df.to_csv(store_name+str(t)+"times"+str(n)+"_gram.csv")

In [261]:
def output_sentence_csv(file, t, s):
    store_name=file.replace("../pretest_", "").replace(".csv", "")
    df = output_df(file,jiku_list,50, t, s)
    df.to_csv(store_name+str(t)+"times_sentence.csv")

In [259]:
for file in store_file:
    output_csv(file,5,2.5,20)

In [263]:
for file in store_file:
    output_sentence_csv(file,1,20)

In [264]:
for file in store_file:
    output_sentence_csv(file,1.5,20)
    output_sentence_csv(file,2,20)
    output_sentence_csv(file,2.5,20)

In [54]:
def n_gram(n, text):
    all_result = []
    texts = text.split("。")
    for text in texts:
        
        a = [("*", "*") for i in range(n)]
        a.extend(tokenize_2(text))
        a.extend([("*", "*") for i in range(n)])
        result = []
        for i in range(n,len(a) - n + 1):
            result.append(a[i-n:i+n+1])
        all_result.extend(result)
    return all_result

In [201]:
def n_gram_result(word, n, review_list):
    pare_time_dic = defaultdict(int)
    if n > 0:
        for text in review_list:
            for gram in n_gram(n,text):
                if word == gram[n][0]:
                    for g in gram:
                        if word != g[0] and g[0] != "*" and (g[1] == "形容詞" or g[1] == "名詞" or g[1] == "動詞"):
                            pare_time_dic[(word, g[0])] += 1
                        #if g[0] != word and g[0] != "*" :
                            #pare_time_dic[(word, g[0])] += 1
    counter = Counter(pare_time_dic)
    common = counter.most_common()
    return common

In [69]:
def n_gram_result_dinner(word, n, file, dinner):
    pare_time_dic = defaultdict(int)
    df = pd.read_csv(file)
    if dinner == True:
        for text in df["dinner_review"]:
            for gram in n_gram(n,text):
                if word == gram[n][0]:
                    for g in gram:
                        #if word != g[0] and g[1] == "形容詞":
                            #pare_time_dic[(word, g[0])] += 1
                        if g[0] != word and g[0] != "*" :
                            pare_time_dic[(word, g[0])] += 1
    counter = Counter(pare_time_dic)
    common = counter.most_common()
    return common

In [206]:
sushi = ["鮨","旨い","握り"]
kome = ["シャリ","握り","シャリ","酢飯","酢","塩",
                  "砂糖","味","香り","硬さ","炊き具合",
                  "温度","米"]
neta = ["ネタ","ネタ","温度","厚さ","薄さ","大きさ","鮮度","熟成",
                 "くさい","香り","生臭い","旨み","甘み","塩",
                 "骨","舌触り","滑らか","パサパサ","漁場","時期",
                 "工夫","締め","名物","魚介","魚","食材",
                 "種類"]
sozai = ["素材単体","マグロ","鮪","大トロ","ヅケマグロ","コハダ",
                 "イワシ","カツオ","アジ","ウニ","雲丹","鯛",
                 "カスゴ鯛","金目鯛","ヒラメ","キス","キンキ","白魚",
                 "白身魚","白身","鮎","鰻","穴子","蛸",
                  "真蛸","イカ","蛍イカ","アカイカ","だるまイカ","車海老",
                  "蟹","毛蟹","鮑","赤貝","トリ貝","牡蠣",
                  "カラスミ","ばちこ","あん肝","玉締め","玉子焼き","玉子",
                  "干瓢","かんぴょう","山葵","つくね芋"]
sozai_sonota = ["素材その他","塩","酢橘","煮キリ","赤酢","煎り酒",
                         "酢味噌","だし","鰹節","ガリ","魚"]
ryouri_sonota = ["料理その他","茶碗蒸し"]
dougu = ["道具","羽釜"]
shigoto = ["仕事","江戸前","仕事","技","技","力","修行","会得", 
               "料理","供す","腕"]
sake = ["酒","酒","ワイン","日本酒","生ビール"]
tsumami = ["つまみ","酒肴","つまみ","刺身","一品料理"]
teikyou = ["提供方法","握り","おまかせ","握る","時間","時","同じ魚",
               "品書き","量","好み","ランチ","昼","夜",
               "コース","最初","最後"]
hito = ["人","主","店主","主人","大将","親方","女将",
            "店員","長男","客","常連","一見","若者",
            "初代","先代","親方","弟子","二番手","仲買人",
            "仕入","仕入先","生産者","業者"]
mise = ["店","つけ場","カウンター","店","店舗","個室","店内",
            "中","内装","外観","入口","広さ","席数",
            "寿司屋","鮨店","訪問","季節感"]
koyutennmei = ["固有店名","かねさか","初音鮨"]
gyoutai = ["業態","営業","予約"]
kakaku = ["価格","高額","控えめ","明朗会計","高い","ランチ",
              "安価","cp","コスパ","コストパフォーマンス","良心的","出費",
              "価格","リーズナブル","予算","お会計","金額","値段",
              "原価率"]
hunniki = ["雰囲気","寿司","満足度","見た目","遅刻","人数",
          "間合い","気持ち","雰囲気","居心地","接客","美味しさ",
          "高級感","エンターテインメント","感想","シーン"]
houmonnbi = ["訪問日","今日","今年","年内","今回","前回","次回",
            "再訪問","再度","今度"]
tochi = ["土地","銀座","四谷","横浜","数寄屋橋","日本橋","中野坂上",
        "京橋","人形町","山梨県","金沢","椎名町","経堂",
        "日比谷","蒲田","立川","都内","地元","この地",
        "産地"]
bunnya = ["分野","日本料理","和食","食べログ","料理"]
kisetsu = ["季節","旬","季節","走り","名残","通年","時期"]
musubi = ["結び","ごちそうさま","感謝","よろしくお願いいたします","ご馳走様でした"]

In [207]:
jiku_list=[sushi,kome,neta,sozai,sozai_sonota,ryouri_sonota,dougu,shigoto,
          sake,tsumami,teikyou,hito,mise,koyutennmei,gyoutai,kakaku,
          hunniki,houmonnbi,tochi,bunnya,kisetsu,musubi]

In [5]:
store_file = ["../pretest_aozora_dinner.csv",
             "../pretest_mitani_lunch.csv",
             "../pretest_mitani_dinner.csv"]

In [188]:
lunch_file = ["../pretest_sushitsuu_lunch.csv",
              "../pretest_sushiichi_lunch.csv",
              "../pretest_sawada_lunch.csv",
              "../pretest_matsukan_lunch.csv",
              "../pretest_kiyoda_lunch.csv",
              "../pretest_imamura_lunch.csv",
              "../pretest_hatsune_lunch.csv"
              ]
dinner_file = ["../pretest_sushitsuu_dinner.csv",
              "../pretest_sushiichi_dinner.csv",
              "../pretest_sawada_dinner.csv",
              "../pretest_matsukan_dinner.csv",
              "../pretest_kiyoda_dinner.csv",
              "../pretest_imamura_dinner.csv",
              "../pretest_hatsune_dinner.csv",
              ]

In [250]:
all_review_word_lunch_2 = defaultdict(int)
all_review_word_dinner_2  =defaultdict(int)
num_word_lunch_2 = 0
num_word_dinner_2 = 0
for file in lunch_file:
    df=pd.read_csv(file)
    for review in df["lunch_review"]:
        list=tokenize(review)
        for l in list:
            all_review_word_lunch_2[l] += 1 #その単語の登場回数を1加える
            num_word_lunch_2 += 1
for file in dinner_file:
    df=pd.read_csv(file)
    for review in df["dinner_review"]:
        list=tokenize(review)
        for l in list:
            all_review_word_dinner_2[l] += 1 #その単語の登場回数を1加える
            num_word_dinner_2 += 1

In [253]:
num_word_lunch_2

103058

In [246]:
len(list)

1088

In [241]:
all_review_word_lunch_2["シャリ"]

9

In [31]:
all_review_df = pd.read_csv("../processed_all_results.csv")
all_review_word_lunch = defaultdict(int) #全てのレビューの登場単語をkey,その登場回数をvalue
                                   #にした辞書
all_review_word_dinner = defaultdict(int)
#all_review_wordを作成していく
#総単語数をnum_wordとする
num_word_lunch = 0
num_word_dinner = 0
for i in range(len(all_review_df)):
    l_review = all_review_df.iloc[i]["lunch_review"] #i番目のlunch_review
    d_review = all_review_df.iloc[i]["dinner_review"] #i番目のdinner_review
    if type(l_review) != float: #nanのときは型がfloatになる
        list = tokenize(l_review) #形態素解析
        for l in list:
            all_review_word_lunch[l] += 1 #その単語の登場回数を1加える
            num_word_lunch += 1
    if type(d_review) != float: #nanのときは型がfloatになる
        list = tokenize(d_review) #形態素解析
        for l in list:
            all_review_word_dinner[l] += 1 #その単語の登場回数を1加える
            num_word_dinner += 1

In [58]:
print("======================================================================")

In [80]:
print("======================================================================")

In [177]:
df3 = pd.read_csv(store_file[0])
results = n_gram_result("シャリ", 8, df3["dinner_review"])
l = [w[0][1] + str(w[1]) for w in results]
l

['強い19',
 '美味しい8',
 '素晴らしい8',
 '高い7',
 '良い5',
 '面白い4',
 '固い3',
 'いい3',
 'ない2',
 'すごい2',
 'よい2',
 '甘い2',
 '浅い2',
 '大きい2',
 '小さい1',
 '嬉しい1',
 '柔らかい1',
 '有難い1',
 'おかしい1',
 '多い1',
 '力強い1',
 'たまらない1',
 '軽い1',
 '淡い1',
 'おこがましい1',
 '硬い1',
 '難しい1',
 '白い1']

In [ ]:
def n_gram_result_dinner(word, n, file, dinner):
    pare_time_dic = defaultdict(int)
    df = pd.read_csv(file)
    if dinner == True:
        for text in df["dinner_review"]:
            for gram in n_gram(n,text):
                if word == gram[n][0]:
                    for g in gram:
                        #if word != g[0] and g[1] == "形容詞":
                            #pare_time_dic[(word, g[0])] += 1
                        if g[0] != word and g[0] != "*" :
                            pare_time_dic[(word, g[0])] += 1
    counter = Counter(pare_time_dic)
    common = counter.most_common()
    return common